In [3]:
import gzip
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import corpora, models, similarities
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel

asins = dict()
categories = dict()
reviewtextlist = list()

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)


## Read categories,asins for a specific category and keep in
## a dictionary
def readMetaData(path,catname):
    i = 0
    for d in parse(path):
        numCategories=0
    #print d['categories'], len(d['categories'][0])
    #print d['title'],d['price'],d['salesrank']
        title= "Not available"
        if 'title' in d:
            title=d['title']

        if(len(d['categories']) > 0):
            numCategories = len(d['categories'][0])
            for i in  range(0,numCategories):
                cat = d['categories'][0][i]
                if cat not in categories:
                    categories[cat]=cat
                if cat == catname:
                    asins[d['asin']] = title
        i += 1

def readReviewdata(path):
    with gzip.open(path, "rb") as f:
        for line in f:
            # print line
            d = eval(line)
            reviewerId = d['reviewerID']
            asin = d['asin']
            rating = d['overall']
            summary = d['summary']
            reviewText = d['reviewText']
            dtime = d['reviewTime']
            # dt = datetime.datetime.strptime(dtime, "%m %d, %Y")
            # if len(d['reviewText']) == 0:
            #     continue
            # if dt.year > year:
            if asin in asins and len (reviewText)>0:
                reviewtextlist.append(reviewText)


def clean(doc):
    lemma = WordNetLemmatizer()
    #stopset = set(stopwords.words('english'))
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopwords.words('english')])
    punc_free = ''.join(ch for ch in stop_free if ch not in string.punctuation)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #return stop_free
    return normalized


def main():
    readMetaData('meta_Cell_Phones_and_Accessories.json.gz',"Headsets")
    #for k, v in asins.iteritems():
    #    print k,v
    readReviewdata('reviews_Cell_Phones_and_Accessories.json.gz')
    reviewId = 0
    reviewtextlist_capped = reviewtextlist[0:10000]
    for ll in reviewtextlist_capped:
        #print reviewId,":",ll
        reviewId +=1

    doc_clean = [clean(reviewText).split() for reviewText in reviewtextlist_capped]
    #print doc_clean
    # Creating the term dictionary of our courpus, where every unique term is assigned an index.
    dictionary = corpora.Dictionary(doc_clean)
    #print(dictionary.token2id)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    #doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    corpus = [dictionary.doc2bow(doc) for doc in doc_clean]
    #print corpus
    tfidf = TfidfModel(corpus)
    #print tfidf[corpus[0]]
    corpus_tfidf = tfidf[corpus]

    n_topics = 10
    #lda = LdaModel(corpus_tfidf, id2word=dictionary, num_topics=n_topics)
    lda = LdaModel(corpus, id2word=dictionary, num_topics=n_topics, passes=100)
    print lda

    for i in range(0, n_topics):
        temp = lda.show_topic(i, 15)
        terms = []
        for term in temp:
            terms.append(term)
        print "Top 15 terms for topic #" + str(i) + ": "+ ", ".join([i[0] for i in terms])

    # document-topic distribution
    for doc in corpus:
        temp = lda.get_document_topics(doc)
    # returns a list of (topic_id, topic_probability) 2-tuples.

    # topic-word distribution
    for i in range(0, n_topics):
        temp = lda.get_topic_terms(i, 15)
    # returns a list of (word_id, probability) 2-tuples for the topn most probable words in topic topicid.

if __name__ == "__main__":
    main()    




LdaModel(num_terms=24948, num_topics=10, decay=0.5, chunksize=2000)
Top 15 terms for topic #0: phone, work, adapter, plug, headphone, jack, 25mm, iphone, music, stereo, cell, product, nokia, listen, 35mm
Top 15 terms for topic #1: noise, wind, xbox, truck, shut, highway, speed, traffic, background, 360, youre, loses, standing, live, razr
Top 15 terms for topic #2: one, product, work, it, bought, headset, would, great, item, amazon, buy, got, new, first, month
Top 15 terms for topic #3: software, dragon, nokia, ipaq, hp, la, 3650, voip, htc, 6600, audio, potential, music, support, amplifier
Top 15 terms for topic #4: headset, phone, it, call, use, bluetooth, time, battery, work, button, one, get, device, unit, good
Top 15 terms for topic #5: ear, fit, sound, it, piece, like, one, headset, earpiece, good, quality, get, well, comfortable, stay
Top 15 terms for topic #6: headset, hear, sound, quality, good, great, it, phone, volume, work, use, bluetooth, noise, well, one
Top 15 terms for t